In [ ]:
# Libraries

import numpy
import os
import platform
import random
import shutil
import sys

In [ ]:
# Ensure source path

ROOT = os.getcwd()

while not ROOT.endswith('upolanc-thesis') :
	ROOT = os.path.abspath(os.path.join(ROOT, os.pardir))

	if len(ROOT) < len('upolanc-thesis') :
		if   platform.system() == 'Linux'   : ROOT = '/d/hpc/projects/FRI/up4472/upolanc-thesis'
		elif platform.system() == 'Windows' : ROOT = 'C:\\Developer\\Workspace\\PyCharm\\Projects\\upolanc-thesis'
		else : raise ValueError()

		print(f'Warning : could not find correct directory, using default : {ROOT}')
		print()

		break

if ROOT not in sys.path :
	sys.path.append(ROOT)

os.chdir(ROOT)

In [ ]:
# Code

from source.python               import runtime
from source.python.data.feature  import feature_extractor
from source.python.data.mutation import mutation_sequence
from source.python.io            import loader
from source.python.io            import writer

runtime.set_numpy_format()
runtime.set_pandas_format()
runtime.set_plot_theme()

# 1. Setup

In [ ]:
# Setup some directory paths

FILTER_ID = 2
SUBFOLDER = 'filter' + str(FILTER_ID)

CWD = ROOT
OUT = os.path.join(CWD, 'output')
RES = os.path.join(CWD, 'resources')

OUT_DATA   = os.path.join(OUT, 'nbp04-feature', SUBFOLDER)
RES_GENOME = os.path.join(RES, 'genome')
RES_NBP01  = os.path.join(OUT, 'nbp01-filter',  SUBFOLDER)
RES_NBP02  = os.path.join(OUT, 'nbp02-anndata', SUBFOLDER)

shutil.rmtree(OUT_DATA, ignore_errors = True)

os.makedirs(OUT_DATA, exist_ok = True)

print(f'     Root Directory : {CWD}')
print(f'   Output Directory : {OUT_DATA}')
print(f' Resource Directory : {RES_GENOME}')
print(f' Resource Directory : {RES_NBP01}')
print()

In [ ]:
# Load the annotated and cleaned data

gene_assembly = loader.load_faidx(
	filename  = os.path.join(RES_GENOME, 'arabidopsis-r36', 'gene-assembly.fa')
)

gene_annotation = loader.load_csv(
	filename   = os.path.join(RES_NBP01, 'gene-annotation.csv'),
	low_memory = False
)

anndata = loader.load_h5ad(
	filename = os.path.join(RES_NBP02, 'arabidopsis-r36.h5ad')
)

filter_dict = loader.load_json(
	filename = os.path.join(RES_NBP01, 'filter.json')
)

In [ ]:
# Filtered transcripts

keep_transcript = filter_dict['data']['keep_transcript']
drop_transcript = filter_dict['data']['drop_transcript']

In [ ]:
# Define the region lengths

lengths = {
	'prom_utr5' : [int( 5000), int(  0)],
	'prom_full' : [int( 5000), int(  0)],
	'prom'      :  int( 1000),
	'utr5'      :  int(  300),
	'cds'       :  int(10000),
	'utr3'      :  int(  350),
	'term'      :  int(  500),
	'term_full' : [int(    0), int(500)]
}

padding = {
	'prom_utr5' : 'left',
	'prom_full' : 'left',
	'prom'      : 'left',
	'utr5'      : 'left',
	'cds'       : 'none',
	'utr3'      : 'left',
	'term'      : 'right',
	'term_full' : 'right'
}

# 2. Transcript Regions

In [ ]:
# Group annotations into regions

regions = feature_extractor.annotation_to_regions(
	annotation = gene_annotation,
	lengths    = lengths
)

print()
print('Gene       : {:5d} / {:5d}'.format(regions['Gene'].nunique(), gene_annotation['Gene'].nunique()))
print('Transcript : {:5d} / {:5d}'.format(regions['Transcript'].nunique(), gene_annotation['Transcript'].nunique()))
print()

In [ ]:
# Define transcription and coding regions sites

regions['UTR_Min'] = regions[['Start', 'End']].min(axis = 1)
regions['UTR_Max'] = regions[['Start', 'End']].max(axis = 1)

regions.drop(columns = ['Start', 'End'])

regions['CDS_Min'] = regions['CDS'].apply(lambda x : numpy.min(x) - 1)
regions['CDS_Max'] = regions['CDS'].apply(lambda x : numpy.max(x) - 1)

In [ ]:
# Compute regions for promoter + utr5

regions['Prom_UTR5'] = None

posx = lambda x : max(x - lengths['prom_utr5'][0] + 0, 1)
posy = lambda x : max(x + lengths['prom_utr5'][1] - 1, 1)

negx = lambda x : max(x - lengths['prom_utr5'][1] + 1, 1)
negy = lambda x : max(x + lengths['prom_utr5'][0] + 0, 1)

regions.loc[regions['Strand'] == '+', 'Prom_UTR5'] = regions[regions['Strand'] == '+']['CDS_Min'].apply(lambda x : [[posx(x), posy(x)]])
regions.loc[regions['Strand'] == '-', 'Prom_UTR5'] = regions[regions['Strand'] == '-']['CDS_Max'].apply(lambda x : [[negx(x), negy(x)]])

regions[['Seq', 'Strand', 'Gene', 'Transcript', 'UTR_Min', 'CDS_Min', 'CDS_Max', 'UTR_Max', 'UTR5_Length', 'CDS_Length', 'UTR3_Length', 'Prom_UTR5']].head()

# 3. Transcript Features

In [ ]:
# Convert the regions into sequences and features

sequences, features = feature_extractor.regions_to_features(
	faidx     = gene_assembly,
	dataframe = regions,
	lengths   = lengths,
	verbose   = False
)

In [ ]:
# Convert the dataframe into a dictionary for ease of use

sequences = sequences.copy()
sequences = sequences.set_index('Transcript', drop = False)
sequences = sequences.rename_axis(None, axis = 'index')
sequences = sequences.to_dict('index')

In [ ]:
# Convert the dataframe into a dictionary for ease of use

features = features.copy()
features = features.set_index('Transcript', drop = False)
features = features.rename_axis(None, axis = 'index')
features = features.to_dict('index')

In [ ]:
# Split to keep and drop

keep_sequences = {k : v for k, v in sequences.items() if k in keep_transcript}
keep_features  = {k : v for k, v in features.items()  if k in keep_transcript}

drop_sequences = {k : v for k, v in sequences.items() if k in drop_transcript}
drop_features  = {k : v for k, v in features.items()  if k in drop_transcript}

## 3.1 Fasta

In [ ]:
# Add a header field with more transcript information

keep_sequences = feature_extractor.sequences_extend_kvpair(
	sequences = keep_sequences,
	regions   = regions,
	header    = '{} | {} | {}:{}-{} | {}'
)

drop_sequences = feature_extractor.sequences_extend_kvpair(
	sequences = drop_sequences,
	regions   = regions,
	header    = '{} | {} | {}:{}-{} | {}'
)

In [ ]:
# Example of a padded strand

transcript = list(keep_sequences.keys())[0]

feature_extractor.print_extracted_sequence(
	transcript = transcript,
	sequences  = keep_sequences,
	space      = True
)

In [ ]:
# Example of a padded strand

transcript = list(drop_sequences.keys())[0]

feature_extractor.print_extracted_sequence(
	transcript = transcript,
	sequences  = drop_sequences,
	space      = True
)

In [ ]:
# Save the transcript region sequences

tuples = [
	('Prom_UTR5', 'promoter-utr5'),
	('Prom_Full', 'promoter-full'),
	('Prom',      'promoter'),
	('UTR5',      'utr5'),
	('CDS',       'cds'),
	('UTR3',      'utr3'),
	('Term',      'terminator'),
	('Term_Full', 'terminator-full')
]

for region, filename in tuples :
	data = {
		item[region]['key'] : item[region]['seq']
		for item in keep_sequences.values()
	}

	data = feature_extractor.pad_multiple(
		sequences = data,
		length    = lengths[region.lower()],
		side      = padding[region.lower()],
		pad_value = None
	)

	writer.write_fasta(
		data     = data,
		filename = os.path.join(OUT_DATA, f'sequences-{filename}-keep.fasta')
	)

for region, filename in tuples :
	data = {
		item[region]['key'] : item[region]['seq']
		for item in drop_sequences.values()
	}

	data = feature_extractor.pad_multiple(
		sequences = data,
		length    = lengths[region.lower()],
		side      = padding[region.lower()],
		pad_value = None
	)

	writer.write_fasta(
		data      = data,
		filename = os.path.join(OUT_DATA, f'sequences-{filename}-drop.fasta')
	)

## 3.2 Sequences

In [ ]:
# Merge transcript regions and pad accordingly

keep_2150, keep_6150 = feature_extractor.merge_and_pad_sequences(
	sequences = keep_sequences,
	lengths   = lengths,
	padding   = padding
)

drop_2150, drop_6150 = feature_extractor.merge_and_pad_sequences(
	sequences = drop_sequences,
	lengths   = lengths,
	padding   = padding
)

In [ ]:
# Display an example of a merged transcript sequence

transcript = list(keep_2150.keys())[0].split(' | ')[0]

feature_extractor.print_padded_sequence(
	transcript = transcript,
	sequences  = keep_2150,
	space      = True
)

In [ ]:
# Display an example of a merged transcript sequence

transcript = list(keep_6150.keys())[0].split(' | ')[0]

feature_extractor.print_padded_sequence(
	transcript = transcript,
	sequences  = keep_6150,
	space      = True
)

In [ ]:
# Save the transcript sequences

writer.write_fasta(
	data     = keep_2150,
	filename = os.path.join(OUT_DATA, f'sequences-2150-keep.fasta')
)

writer.write_fasta(
	data     = drop_2150,
	filename = os.path.join(OUT_DATA, f'sequences-2150-drop.fasta')
)

writer.write_fasta(
	data     = keep_6150,
	filename = os.path.join(OUT_DATA, f'sequences-6150-keep.fasta')
)

writer.write_fasta(
	data     = drop_6150,
	filename = os.path.join(OUT_DATA, f'sequences-6150-drop.fasta')
)

## 3.3 Mutations

In [ ]:
# Select random transcripts to mutate

mutation_transcripts = random.choices(list(keep_sequences.keys()), k = 25)
mutation_transcripts = {key : value for key, value in keep_sequences.items() if key in mutation_transcripts}

In [ ]:
# Mutate transcripts multiple times

rates = [
	0.01,
	0.05,
	0.10,
	0.15,
	0.25
]

params = {
	'mutation_rate'     : 0.1,
	'insertion_rate'    : 0.0,
	'deletion_rate'     : 0.0,
	'substitution_rate' : 1.0,
	'max_length'        : 6
}

result = mutation_sequence.generate_multi(
	sequences = mutation_transcripts,
	variants  = 20,
	method    = 'random',
	rates     = rates,
	params    = params,
	verbose   = False
)

mutation_sequences = result[0]
mutation_features  = result[1]

In [ ]:
# Merge mutation transcript regions and pad accordingly

mutation_2150, mutation_6150 = feature_extractor.merge_and_pad_sequences(
	sequences = mutation_sequences,
	lengths   = lengths,
	padding   = padding
)

In [ ]:
# Compute similiarity betwen orginal and mutated transcript sequences

data_2150 = dict()
data_6150 = dict()

for key, value in mutation_2150.items() :
	splits = key.split(' | ')

	orgkey = splits[0].split('-')[0]
	orgkey = orgkey + ' | ' + ' | '.join(splits[1:])

	orgseq = keep_2150[orgkey]

	match = sum([1 if x == y else 0 for x, y in zip(orgseq, value)])
	match = match / len(orgseq)

	data_2150['{} | {:.5f}'.format(key, match)] = value

for key, value in mutation_6150.items() :
	splits = key.split(' | ')

	orgkey = splits[0].split('-')[0]
	orgkey = orgkey + ' | ' + ' | '.join(splits[1:])

	orgseq = keep_6150[orgkey]

	match = sum([1 if x == y else 0 for x, y in zip(orgseq, value)])
	match = match / len(orgseq)

	data_6150['{} | {:.5f}'.format(key, match)] = value

mutation_2150 = data_2150
mutation_6150 = data_6150

In [ ]:
# Save the transcript region sequences

tuples = [
	('Prom_UTR5', 'promoter-utr5'),
	('Prom_Full', 'promoter-full'),
	('Prom',      'promoter'),
	('UTR5',      'utr5'),
	('CDS',       'cds'),
	('UTR3',      'utr3'),
	('Term',      'terminator'),
	('Term_Full', 'terminator-full')
]

createkey = lambda x, y : x + ' | ' + ' | '.join(y['key'].split(' | ')[1:])

for region, filename in tuples :
	data = {
		createkey(key, item[region]) : item[region]['seq']
		for key, item in mutation_sequences.items()
	}

	data = feature_extractor.pad_multiple(
		sequences = data,
		length    = lengths[region.lower()],
		side      = padding[region.lower()],
		pad_value = None
	)

	writer.write_fasta(
		data     = data,
		filename = os.path.join(OUT_DATA, f'mutation-sequences-{filename}.fasta')
	)

In [ ]:
# Save the mutation transcript sequences

writer.write_fasta(
	data     = mutation_2150,
	filename = os.path.join(OUT_DATA, f'mutation-sequences-2150.fasta')
)

writer.write_fasta(
	data     = mutation_6150,
	filename = os.path.join(OUT_DATA, f'mutation-sequences-6150.fasta')
)

In [ ]:
# Extract mutation features

mutation_features_frequency = {
	key : numpy.array(value['Frequency'])
	for key, value in mutation_features.items()
}

mutation_features_stability = {
	key : numpy.array(value['Stability'])
	for key, value in mutation_features.items()
}

In [ ]:
# Save the mutation features

writer.write_npz(
	data     = mutation_features_frequency,
	filename = os.path.join(OUT_DATA, 'mutation-features-frequency')
)

writer.write_npz(
	data     = mutation_features_stability,
	filename = os.path.join(OUT_DATA, 'mutation-features-stability')
)

In [ ]:
# Save merged features

mutation_features_base = dict()

for key in mutation_features_frequency.keys() :
	freq = mutation_features_frequency[key]
	stab = mutation_features_stability[key]

	mutation_features_base[key] = numpy.concatenate((freq, stab), axis = 0)

writer.write_npz(
	data     = mutation_features_base,
	filename = os.path.join(OUT_DATA, 'mutation-features-base')
)

## 3.4 Features

In [ ]:
# Extract features

keep_features_frequency = {
	key : numpy.array(value['Frequency'])
	for key, value in keep_features.items()
}

keep_features_stability = {
	key : numpy.array(value['Stability'])
	for key, value in keep_features.items()
}

drop_features_frequency = {
	key : numpy.array(value['Frequency'])
	for key, value in drop_features.items()
}

drop_features_stability = {
	key : numpy.array(value['Stability'])
	for key, value in drop_features.items()
}

In [ ]:
# Save the features

writer.write_npz(
	data     = keep_features_frequency,
	filename = os.path.join(OUT_DATA, 'features-frequency-keep')
)

writer.write_npz(
	data     = keep_features_stability,
	filename = os.path.join(OUT_DATA, 'features-stability-keep')
)

writer.write_npz(
	data     = drop_features_frequency,
	filename = os.path.join(OUT_DATA, 'features-frequency-drop')
)

writer.write_npz(
	data     = drop_features_stability,
	filename = os.path.join(OUT_DATA, 'features-stability-drop')
)

In [ ]:
# Save merged features

keep_features_base = dict()
drop_features_base = dict()

for key in keep_features_frequency.keys() :
	freq = keep_features_frequency[key]
	stab = keep_features_stability[key]

	keep_features_base[key] = numpy.concatenate((freq, stab), axis = 0)

writer.write_npz(
	data     = keep_features_base,
	filename = os.path.join(OUT_DATA, 'features-base-keep')
)

for key in drop_features_frequency.keys() :
	freq = drop_features_frequency[key]
	stab = drop_features_stability[key]

	drop_features_base[key] = numpy.concatenate((freq, stab), axis = 0)

writer.write_npz(
	data     = drop_features_base,
	filename = os.path.join(OUT_DATA, 'features-base-drop')
)

## 3.5 Anndata

In [ ]:
# Save the annotated data with multiple layers

writer.write_h5ad(
	data     = anndata[:, list(features.keys())].copy(),
	filename = os.path.join(OUT_DATA, 'arabidopsis-r36.h5ad')
)